In [ ]:
# запускать после spmm-inference.ipynb чтоб были все библиотеки и веса

In [1]:
!cd SPMM && python "smiles2emb.py" --input_file "../data/quantitive_neftekod25_data_filter.csv" --seed 42 --checkpoint "../checkpoint_SPMM.ckpt" --device "cpu"

seed: 42
Loading CSV data
Creating model
LOADING PRETRAINED MODEL..
load checkpoint from ../checkpoint_SPMM.ckpt
_IncompatibleKeys(missing_keys=[], unexpected_keys=['temp'])
Generating SMILES embeddings...
Processed 13/13 SMILES
CSV with embeddings saved to smiles_with_embeddings.csv


In [5]:
!pip install catboost

  Using cached catboost-1.2.7-cp310-cp310-win_amd64.whl.metadata (1.2 kB)
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB 163.8 kB/s eta 0:10:21
   ---------------------------------------- 0.0/101.8 MB 163.8 kB/s eta 0:10:21
   ---------------------------------------- 0.0/101.8 MB 163.8 kB/s eta 0:10:21
   ---------------------------------------- 0.0/101.8 MB 115.9 kB/s eta 0:14:38
   ---------------------------------------- 0.0/101.8 MB 115.9 kB/s eta 0:14:38
   ---------------------------------------- 0.0/101.8 MB 115.9 kB/s eta 0:14:38
   -----------------


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   --------------------- ------------------ 55.5/101.8 MB 1.7 MB/s eta 0:00:27
   --------------------- ------------------ 55.6/101.8 MB 1.8 MB/s eta 0:00:27
   --------------------- ------------------ 55.7/101.8 MB 1.7 MB/s eta 0:00:27
   --------------------- ------------------ 55.8/101.8 MB 1.8 MB/s eta 0:00:26
   --------------------- ------------------ 55.9/101.8 MB 1.8 MB/s eta 0:00:27
   --------------------- ------------------ 55.9/101.8 MB 1.8 MB/s eta 0:00:27
   --------------------- ------------------ 55.9/101.8 MB 1.8 MB/s eta 0:00:27
   ---------------------- ----------------- 56.0/101.8 MB 1.8 MB/s eta 0:00:26
   ---------------------- ----------------- 56.1/101.8 MB 1.8 MB/s eta 0:00:26
   ---------------------- ----------------- 56.2/101.8 MB 1.8 MB/s eta 0:00:26
   ---------------------- ----------------- 56.3/101.8 MB 1.8 MB/s eta 0:00:26
   ---------------------- ----------------- 56.3/101.8 MB 1.8 MB/s eta 0:00:26
   ---------------------- ----------------- 56.4/101

In [67]:
import os
import joblib
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
data = pd.read_csv('SPMM/smiles_with_embeddings.csv')
data['emb'] = data['emb'].apply(lambda x: eval(x))

In [4]:
len(data.iloc[0].emb)

768

In [5]:
os.makedirs('models_cb', exist_ok=True)

emb_df = pd.DataFrame(data['emb'].tolist())
emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]
X = pd.concat([emb_df, data[['C']]], axis=1)

y = data['PDSC']

kf = KFold(n_splits=len(data), shuffle=True, random_state=42)
oof_predictions_cb = np.zeros(len(data))

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_cb = CatBoostRegressor(learning_rate=0.01, verbose=0, random_seed=42)
    model_cb.fit(X_train, y_train)

    oof_predictions_cb[test_index] = model_cb.predict(X_test)
    
    model_path = f'models_cb/catboost_model_fold_{fold}.joblib'
    joblib.dump(model_cb, model_path)

In [6]:
mse = mean_squared_error(y, oof_predictions_cb)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, oof_predictions_cb)
r2 = r2_score(y, oof_predictions_cb)

print(f'''
CatBoost Metrics
- MSE: {mse:.4f}
- RMSE: {rmse:.4f}
- MAE: {mae:.4f}
- R²: {r2:.4f}
''')


CatBoost Metrics
- MSE: 905.3852
- RMSE: 30.0896
- MAE: 22.4417
- R²: -0.1156



In [7]:
os.makedirs('models_cb_pca', exist_ok=True)

emb_df = pd.DataFrame(data['emb'].tolist())
emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]

pca = PCA(n_components=5)
pca_embeddings = pca.fit_transform(emb_df)

pca_emb_df = pd.DataFrame(pca_embeddings, columns=[f'pca_emb_{i}' for i in range(pca_embeddings.shape[1])])

X = pd.concat([pca_emb_df, data[['C']]], axis=1)

y = data['PDSC']

kf = KFold(n_splits=len(data), shuffle=True, random_state=42)
oof_predictions_cb_pca = np.zeros(len(data))

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_cb_pca = CatBoostRegressor(learning_rate=0.01, verbose=0, random_seed=42)
    model_cb_pca.fit(X_train, y_train)

    oof_predictions_cb_pca[test_index] = model_cb_pca.predict(X_test)
    
    model_path = f'models_cb_pca/catboost_pca_model_fold_{fold}.joblib'
    joblib.dump(model_cb_pca, model_path)

In [8]:
mse = mean_squared_error(y, oof_predictions_cb_pca)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, oof_predictions_cb_pca)
r2 = r2_score(y, oof_predictions_cb_pca)

print(f'''
CatBoost PCA Metrics
- MSE: {mse:.4f}
- RMSE: {rmse:.4f}
- MAE: {mae:.4f}
- R²: {r2:.4f}
''')


CatBoost PCA Metrics
- MSE: 782.4317
- RMSE: 27.9720
- MAE: 19.9905
- R²: 0.0359



In [9]:
os.makedirs('models_ridge', exist_ok=True)

emb_df = pd.DataFrame(data['emb'].tolist())
emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]
X = pd.concat([emb_df, data[['C']]], axis=1)

y = data['PDSC']

kf = KFold(n_splits=len(data), shuffle=True, random_state=42)
oof_predictions_ridge = np.zeros(len(data))

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_ridge = Ridge(random_state=42)
    model_ridge.fit(X_train, y_train)

    oof_predictions_ridge[test_index] = model_ridge.predict(X_test)

    model_path = f'models_ridge/ridge_model_fold_{fold}.joblib'
    joblib.dump(model_ridge, model_path)

In [10]:
mse = mean_squared_error(y, oof_predictions_ridge)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, oof_predictions_ridge)
r2 = r2_score(y, oof_predictions_ridge)

print(f'''
Ridge Metrics
- MSE: {mse:.4f}
- RMSE: {rmse:.4f}
- MAE: {mae:.4f}
- R²: {r2:.4f}
''')


Ridge Metrics
- MSE: 700.5420
- RMSE: 26.4678
- MAE: 18.2600
- R²: 0.1368



In [17]:
os.makedirs('models_ridge_pca', exist_ok=True)
os.makedirs('models_pca', exist_ok=True)

emb_df = pd.DataFrame(data['emb'].tolist())
emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]

pca = PCA(n_components=5)
pca_embeddings = pca.fit_transform(emb_df)

joblib.dump(pca, 'models_pca/pca_model.joblib')

pca_emb_df = pd.DataFrame(pca_embeddings, columns=[f'pca_emb_{i}' for i in range(pca_embeddings.shape[1])])

X = pd.concat([pca_emb_df, data[['C']]], axis=1)

y = data['PDSC']

kf = KFold(n_splits=len(data), shuffle=True, random_state=42)
oof_predictions_ridge_pca = np.zeros(len(data))

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_ridge_pca = Ridge(random_state=42)
    model_ridge_pca.fit(X_train, y_train)

    oof_predictions_ridge_pca[test_index] = model_ridge_pca.predict(X_test)

    model_path = f'models_ridge_pca/ridge_pca_model_fold_{fold}.joblib'
    joblib.dump(model_ridge_pca, model_path)

In [18]:
mse = mean_squared_error(y, oof_predictions_ridge_pca)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, oof_predictions_ridge_pca)
r2 = r2_score(y, oof_predictions_ridge_pca)

print(f'''
Ridge PCA Metrics
- MSE: {mse:.4f}
- RMSE: {rmse:.4f}
- MAE: {mae:.4f}
- R²: {r2:.4f}
''')


Ridge PCA Metrics
- MSE: 532.7240
- RMSE: 23.0808
- MAE: 16.1426
- R²: 0.3436



In [13]:
os.makedirs('SPMM/valid_molecules_csv/', exist_ok=True)

for gen_data_path in os.listdir('SPMM/valid_molecules/'):
    gen_data = pd.read_csv('SPMM/valid_molecules/' + gen_data_path, header=None).rename(columns={0: 'Smiles'})
    gen_data.to_csv('SPMM/valid_molecules_csv/' + gen_data_path.replace('txt', 'csv'), index=False)

In [15]:
import glob

os.makedirs('SPMM/valid_molecules_with_emb_csv/', exist_ok=True)
directory = "SPMM/valid_molecules_csv"

for csv_file in glob.glob(os.path.join(directory, "*.csv")):
    csv_file = csv_file.replace('\\', '/').replace('SPMM/', '')
    output_file = csv_file.replace('valid_molecules_csv', 'valid_molecules_with_emb_csv').replace('.csv', '_with_emb.csv')
    print(f"{csv_file=}, {output_file=}")
    !cd SPMM && python "smiles2emb.py" --input_file "{csv_file}" --output_file "{output_file}" --seed 42 --checkpoint "../checkpoint_SPMM.ckpt" --device "cpu"

csv_file='valid_molecules_csv/valid_generated_molecules_0.csv', output_file='valid_molecules_with_emb_csv/valid_generated_molecules_0_with_emb.csv'
seed: 42
Loading CSV data
Creating model
LOADING PRETRAINED MODEL..
load checkpoint from ../checkpoint_SPMM.ckpt
_IncompatibleKeys(missing_keys=[], unexpected_keys=['temp'])
Generating SMILES embeddings...
Processed 94/94 SMILES
CSV with embeddings saved to valid_molecules_with_emb_csv/valid_generated_molecules_0_with_emb.csv
csv_file='valid_molecules_csv/valid_generated_molecules_1.csv', output_file='valid_molecules_with_emb_csv/valid_generated_molecules_1_with_emb.csv'
seed: 42
Loading CSV data
Creating model
LOADING PRETRAINED MODEL..
load checkpoint from ../checkpoint_SPMM.ckpt
_IncompatibleKeys(missing_keys=[], unexpected_keys=['temp'])
Generating SMILES embeddings...
Processed 99/99 SMILES
CSV with embeddings saved to valid_molecules_with_emb_csv/valid_generated_molecules_1_with_emb.csv
csv_file='valid_molecules_csv/valid_generated_mo

In [12]:
from rdkit import Chem

folder_path = 'SPMM/valid_molecules_with_emb_csv'
output_path = 'SPMM/valid_molecules_with_emb_csv_PDSC/'
models_path = 'models_ridge_pca'
pca_path = 'models_pca/pca_model.joblib'

os.makedirs(output_path, exist_ok=True)

model_files = [f for f in os.listdir(models_path) if f.endswith('.joblib')]
models = []
for model_file in model_files:
    model_path = os.path.join(models_path, model_file)
    model = joblib.load(model_path)
    models.append(model)

pca = joblib.load(pca_path)

In [87]:
def assign_flag(smiles):
    mol = Chem.MolFromSmiles(smiles)

    aromatic_amine_pattern = Chem.MolFromSmarts('c-[NH2,NH1,NH0;!$(NC=O);!$(N=*);!$(N#*)]')
    phenol_pattern = Chem.MolFromSmarts('c-[OH]')
    
    has_aromatic_amine = mol.HasSubstructMatch(aromatic_amine_pattern)
    has_phenol = mol.HasSubstructMatch(phenol_pattern)
    
    if has_aromatic_amine:
        return 'aromaamin'
    elif has_phenol:
        return 'fenol'
    return 'none'

def assign_c(flag):
    if flag == 'aromaamin':
        return 0.05
    elif flag == 'fenol':
        return 0.1
    else:
        return 0

In [81]:
def prepare_for_inference(df):
    emb_df = pd.DataFrame(df['emb'].tolist())
    emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]

    pca_embeddings = pca.transform(emb_df)
 
    pca_emb_df = pd.DataFrame(
        pca_embeddings, 
        columns=[f'pca_emb_{i}' for i in range(pca_embeddings.shape[1])]
    )
    
    X = pd.concat([pca_emb_df, df[['C']]], axis=1)
    
    return X

In [17]:
from tqdm import tqdm

for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.csv') and file_name.startswith('valid_generated_molecules_'):
        file_path = os.path.join(folder_path, file_name)
        
        df = pd.read_csv(file_path)
        df['emb'] = df['emb'].apply(lambda x: eval(x))
        
        df['flag'] = df['Smiles'].apply(assign_flag)
        df['C'] = df['flag'].apply(assign_c)

        # print(df[df['flag'] == 'none'].iloc[0].Smiles)
        df = df[df['flag'] != 'none']
        df = df.reset_index(drop=True)
        
        X = prepare_for_inference(df)
        
        all_predictions = []
        for model in models:
            pred = model.predict(X)
            all_predictions.append(pred)
        
        df['PDSC'] = np.mean(all_predictions, axis=0)
        
        output_file_name = f"processed_{file_name}"
        output_file_path = os.path.join(output_path, output_file_name)
        df.drop(columns=['emb'], axis=1, inplace=True)
        df.to_csv(output_file_path, index=False)
        
        # print(f"processed {file_name=}, saved to {output_file_name=}")

100%|██████████| 13/13 [00:03<00:00,  3.82it/s]


In [18]:
dfs = []
for file in os.listdir('SPMM/valid_molecules_with_emb_csv_PDSC/'):
    curr_df = pd.read_csv('SPMM/valid_molecules_with_emb_csv_PDSC/' + file)
    dfs.append(curr_df)

In [19]:
df_final = pd.concat(dfs, axis=0)

In [20]:
df_final = df_final.sort_values(by='PDSC', ascending=False)

In [35]:
pd.DataFrame(df_final.head(60)['Smiles']).to_csv('submit.csv', index=False)

In [36]:
lb_prob = pd.read_csv('data/lb_prob.csv')
df_final60 = df_final.head(60)

In [46]:
merged_df = pd.merge(df_final60, lb_prob, on='Smiles', how='left')

In [52]:
new_train = merged_df[['Smiles', 'PDSC_y']].rename(columns={'PDSC_y': 'PDSC'})

In [53]:
bad_data = new_train[pd.isna(new_train['PDSC'])]

In [55]:
good_data = new_train[~pd.isna(new_train['PDSC'])]

In [58]:
good_data.to_csv('data/lb_prob_filter.csv', index=False)

In [59]:
!cd SPMM && python "smiles2emb.py" --input_file "../data/lb_prob_filter.csv" --output_file "smiles_with_embeddings_lb_prob.csv" --seed 42 --checkpoint "../checkpoint_SPMM.ckpt" --device "cpu"

seed: 42
Loading CSV data
Creating model
LOADING PRETRAINED MODEL..
load checkpoint from ../checkpoint_SPMM.ckpt
_IncompatibleKeys(missing_keys=[], unexpected_keys=['temp'])
Generating SMILES embeddings...
Processed 38/38 SMILES
CSV with embeddings saved to smiles_with_embeddings_lb_prob.csv


In [65]:
data = pd.read_csv('SPMM/smiles_with_embeddings_lb_prob.csv')
data['emb'] = data['emb'].apply(lambda x: eval(x))

In [78]:
os.makedirs('models_ridge_pca_lb_prob', exist_ok=True)
os.makedirs('models_pca_lb_prob', exist_ok=True)

emb_df = pd.DataFrame(data['emb'].tolist())
emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]

pca = PCA(n_components=5)
pca_embeddings = pca.fit_transform(emb_df)

joblib.dump(pca, 'models_pca_lb_prob/pca_model_lb_prob.joblib')

pca_emb_df = pd.DataFrame(pca_embeddings, columns=[f'pca_emb_{i}' for i in range(pca_embeddings.shape[1])])

X = pca_emb_df
y = data['PDSC']

kf = KFold(n_splits=len(data), shuffle=True, random_state=42)
oof_predictions_ridge_pca_lb_prob = np.zeros(len(data))

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_ridge_pca_lb_prob = Ridge(random_state=42)
    model_ridge_pca_lb_prob.fit(X_train, y_train)

    oof_predictions_ridge_pca_lb_prob[test_index] = model_ridge_pca_lb_prob.predict(X_test)

    model_path = f'models_ridge_pca_lb_prob/ridge_pca_lb_prob_model_fold_{fold}.joblib'
    joblib.dump(model_ridge_pca_lb_prob, model_path)

In [79]:
mse = mean_squared_error(y, oof_predictions_ridge_pca_lb_prob)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, oof_predictions_ridge_pca_lb_prob)
r2 = r2_score(y, oof_predictions_ridge_pca_lb_prob)

print(f'''
Ridge PCA LB Prob Metrics
- MSE: {mse:.4f}
- RMSE: {rmse:.4f}
- MAE: {mae:.4f}
- R²: {r2:.4f}
''')


Ridge PCA LB Prob Metrics
- MSE: 432.0170
- RMSE: 20.7850
- MAE: 16.7442
- R²: 0.1557



In [84]:
from rdkit import Chem

folder_path = 'SPMM/valid_molecules_with_emb_csv'
output_path = 'SPMM/valid_molecules_with_emb_csv_LBPROB_PDSC'
models_path = 'models_ridge_pca_lb_prob'
pca_path = 'models_pca_lb_prob/pca_model_lb_prob.joblib'

os.makedirs(output_path, exist_ok=True)

model_files = [f for f in os.listdir(models_path) if f.endswith('.joblib')]
models = []
for model_file in model_files:
    model_path = os.path.join(models_path, model_file)
    model = joblib.load(model_path)
    models.append(model)

pca = joblib.load(pca_path)

In [127]:
def prepare_for_inference(df):
    emb_df = pd.DataFrame(df['emb'].tolist())
    emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]

    pca_embeddings = pca.transform(emb_df)
 
    pca_emb_df = pd.DataFrame(
        pca_embeddings, 
        columns=[f'pca_emb_{i}' for i in range(pca_embeddings.shape[1])]
    )
    
    X = pca_emb_df
    
    return X

In [88]:
from tqdm import tqdm

for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.csv') and file_name.startswith('valid_generated_molecules_'):
        file_path = os.path.join(folder_path, file_name)
        
        df = pd.read_csv(file_path)
        df['emb'] = df['emb'].apply(lambda x: eval(x))
        
        df['flag'] = df['Smiles'].apply(assign_flag)

        df = df[df['flag'] != 'none']
        df = df.reset_index(drop=True)
        
        X = prepare_for_inference(df)
        
        all_predictions = []
        for model in models:
            pred = model.predict(X)
            all_predictions.append(pred)
        
        df['PDSC'] = np.mean(all_predictions, axis=0)
        
        output_file_name = f"processed_{file_name}"
        output_file_path = os.path.join(output_path, output_file_name)
        df.drop(columns=['emb'], axis=1, inplace=True)
        df.to_csv(output_file_path, index=False)
        
        # print(f"processed {file_name=}, saved to {output_file_name=}")

100%|██████████| 13/13 [00:03<00:00,  3.64it/s]


In [102]:
dfs = []
for file in os.listdir('SPMM/valid_molecules_with_emb_csv_LBPROB_PDSC/'):
    curr_df = pd.read_csv('SPMM/valid_molecules_with_emb_csv_LBPROB_PDSC/' + file)
    dfs.append(curr_df)

In [103]:
df_final = pd.concat(dfs, axis=0)

In [104]:
df_final = df_final.sort_values(by='PDSC', ascending=False)

In [106]:
used = new_train.Smiles.tolist()

In [107]:
df_final = df_final[~df_final['Smiles'].isin(used)]

In [109]:
pd.DataFrame(df_final.head(40)['Smiles']).to_csv('submit.csv', index=False)

In [110]:
lb_prob2 = pd.read_csv('data/lb_prob2.csv')
df_final40 = df_final.head(40)

In [111]:
merged_df = pd.merge(df_final40, lb_prob2, on='Smiles', how='left')

In [113]:
new_train2 = merged_df[['Smiles', 'PDSC_y']].rename(columns={'PDSC_y': 'PDSC'})

In [114]:
bad_data2 = new_train2[pd.isna(new_train2['PDSC'])]

In [115]:
good_data2 = new_train2[~pd.isna(new_train2['PDSC'])]

In [119]:
good_data2.to_csv('data/lb_prob2_filter.csv', index=False)

In [121]:
all_good_data = pd.concat([good_data, good_data2], axis=0)
all_good_data.to_csv('data/lb_prob_1and2_filter.csv', index=False)

In [122]:
!cd SPMM && python "smiles2emb.py" --input_file "../data/lb_prob_1and2_filter.csv" --output_file "smiles_with_embeddings_lb_prob2.csv" --seed 42 --checkpoint "../checkpoint_SPMM.ckpt" --device "cpu"

seed: 42
Loading CSV data
Creating model
LOADING PRETRAINED MODEL..
load checkpoint from ../checkpoint_SPMM.ckpt
_IncompatibleKeys(missing_keys=[], unexpected_keys=['temp'])
Generating SMILES embeddings...
Processed 63/63 SMILES
CSV with embeddings saved to smiles_with_embeddings_lb_prob2.csv


In [123]:
data = pd.read_csv('SPMM/smiles_with_embeddings_lb_prob2.csv')
data['emb'] = data['emb'].apply(lambda x: eval(x))

In [124]:
os.makedirs('models_ridge_pca_lb_prob2', exist_ok=True)
os.makedirs('models_pca_lb_prob2', exist_ok=True)

emb_df = pd.DataFrame(data['emb'].tolist())
emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]

pca = PCA(n_components=5)
pca_embeddings = pca.fit_transform(emb_df)

joblib.dump(pca, 'models_pca_lb_prob2/pca_model_lb_prob2.joblib')

pca_emb_df = pd.DataFrame(pca_embeddings, columns=[f'pca_emb_{i}' for i in range(pca_embeddings.shape[1])])

X = pca_emb_df
y = data['PDSC']

kf = KFold(n_splits=len(data), shuffle=True, random_state=42)
oof_predictions_ridge_pca_lb_prob2 = np.zeros(len(data))

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_ridge_pca_lb_prob2 = Ridge(random_state=42)
    model_ridge_pca_lb_prob2.fit(X_train, y_train)

    oof_predictions_ridge_pca_lb_prob2[test_index] = model_ridge_pca_lb_prob2.predict(X_test)

    model_path = f'models_ridge_pca_lb_prob2/ridge_pca_lb_prob2_model_fold_{fold}.joblib'
    joblib.dump(model_ridge_pca_lb_prob2, model_path)

In [125]:
mse = mean_squared_error(y, oof_predictions_ridge_pca_lb_prob2)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, oof_predictions_ridge_pca_lb_prob2)
r2 = r2_score(y, oof_predictions_ridge_pca_lb_prob2)

print(f'''
Ridge PCA LB Prob2 Metrics
- MSE: {mse:.4f}
- RMSE: {rmse:.4f}
- MAE: {mae:.4f}
- R²: {r2:.4f}
''')


Ridge PCA LB Prob2 Metrics
- MSE: 343.0037
- RMSE: 18.5204
- MAE: 15.1146
- R²: 0.1328



In [126]:
from rdkit import Chem

folder_path = 'SPMM/valid_molecules_with_emb_csv'
output_path = 'SPMM/valid_molecules_with_emb_csv_LBPROB2_PDSC'
models_path = 'models_ridge_pca_lb_prob2'
pca_path = 'models_pca_lb_prob2/pca_model_lb_prob2.joblib'

os.makedirs(output_path, exist_ok=True)

model_files = [f for f in os.listdir(models_path) if f.endswith('.joblib')]
models = []
for model_file in model_files:
    model_path = os.path.join(models_path, model_file)
    model = joblib.load(model_path)
    models.append(model)

pca = joblib.load(pca_path)

In [128]:
from tqdm import tqdm

for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.csv') and file_name.startswith('valid_generated_molecules_'):
        file_path = os.path.join(folder_path, file_name)
        
        df = pd.read_csv(file_path)
        df['emb'] = df['emb'].apply(lambda x: eval(x))
        
        df['flag'] = df['Smiles'].apply(assign_flag)

        df = df[df['flag'] != 'none']
        df = df.reset_index(drop=True)
        
        X = prepare_for_inference(df)
        
        all_predictions = []
        for model in models:
            pred = model.predict(X)
            all_predictions.append(pred)
        
        df['PDSC'] = np.mean(all_predictions, axis=0)
        
        output_file_name = f"processed_{file_name}"
        output_file_path = os.path.join(output_path, output_file_name)
        df.drop(columns=['emb'], axis=1, inplace=True)
        df.to_csv(output_file_path, index=False)
        
        # print(f"processed {file_name=}, saved to {output_file_name=}")

100%|██████████| 13/13 [00:03<00:00,  3.38it/s]


In [129]:
dfs = []
for file in os.listdir('SPMM/valid_molecules_with_emb_csv_LBPROB2_PDSC/'):
    curr_df = pd.read_csv('SPMM/valid_molecules_with_emb_csv_LBPROB2_PDSC/' + file)
    dfs.append(curr_df)

In [130]:
df_final = pd.concat(dfs, axis=0)

In [131]:
df_final = df_final.sort_values(by='PDSC', ascending=False)

In [136]:
new_train12 = pd.concat([new_train, new_train2], axis=0)

In [137]:
used = new_train12.Smiles.tolist()

In [138]:
df_final = df_final[~df_final['Smiles'].isin(used)]

In [139]:
df_final

,Smiles,flag,PDSC
2,CCCN(CCC)c1ccc(-c2ccc(CNCCO)cc2)cc1,aromaamin,41.153501
8,CCCCN(CCCC)c1ccc(-c2ccc(CNCCO)cc2)cc1,aromaamin,41.098296
11,CCCNC(Cc1ccc(N(CC)CC)cc1)c1cccc2ccccc12,aromaamin,40.623814
15,CCCCN(CCCC)c1ccc(-c2ccc(NCCO)cc2)cc1,aromaamin,40.196503
56,CCCCCCN(CCCCC)c1cc2ccccc2cc1CNCCC,aromaamin,39.597667
...,...,...,...
20,Cc1ccnc(C)c1O,fenol,4.715081
29,C=CCc1ccc(O)cn1,fenol,3.927001
8,Cc1nc(C)ccc1O,fenol,3.664151
7,C#Cc1ccc(C)c(O)c1,fenol,3.036398
